In [1]:
import torch
import json_repair
import pandas as pd

from tqdm import tqdm
from datasets import Dataset

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported

from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    #model_name="unsloth/Qwen2.5-7B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.47.0.
   \\   /|    GPU: NVIDIA RTX A6000. Max memory: 47.529 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"], 
    use_rslora=True,
    use_gradient_checkpointing=True
)

Unsloth 2024.12.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## data

In [4]:
df = pd.read_csv("/Utilisateurs/umushtaq/emotion_analysis_comics/dataset_files/comics_dataset_pg.csv")

In [5]:
df

,Unnamed: 0,file_name,page_nr,split,utterance,emotion_c
0,0,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1,TRAIN,"[""THIS VILE THING ATTACKED THE SMALL BEASTS OF...","[['anger'], ['anger'], ['fear'], ['fear'], ['f..."
1,1,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,2,TRAIN,"[""NO-- #GKKK\u2026#"", ""#CHOMP!"", ""BY THE SKIN...","[['fear'], ['anger'], ['surprise'], ['anger'],..."
2,2,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,3,TRAIN,"[""COME ON, BEAST!"", ""SHOW YOURSELF!"", ""WHY DO ...","[['joy'], ['joy'], ['anger'], ['anger']]"
3,3,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,4,TRAIN,"[""#AARGH! ""]","[['fear', 'surprise']]"
4,4,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,5,TRAIN,"[""I, THE GREEN TORCH, HAVE BEEN TASKED WITH PR...","[['anger'], ['anger'], ['fear'], ['fear', 'sur..."
...,...,...,...,...,...,...
869,869,QC copy - 2200 - Stillwater 13.xlsx,16,TEST,"[""WE WERE IN GALEN'S OFFICE. YOU WERE ABOUT TO...","[['anger'], ['anger'], ['anger'], ['anger'], [..."
870,870,QC copy - 2200 - Stillwater 13.xlsx,17,TEST,"[""SO WHAT ARE WE GOING TO DO?"", ""THE WAY I SEE...","[['sadness', 'surprise'], ['anger'], ['anger']..."
871,871,QC copy - 2200 - Stillwater 13.xlsx,18,TEST,"[""KIDDIE COUNCIL'S BEEN GOING A LONG TIME... ""...","[['anger', 'sadness'], ['anger'], ['anger'], [..."
872,872,QC copy - 2200 - Stillwater 13.xlsx,19,TEST,"[""IT'S BEEN\u2026 PEACEFUL. ASIDE FROM SHIT LI...","[['anger'], ['joy'], ['joy'], ['anger', 'surpr..."


In [6]:
def get_nr_emotions(row):

    return len(row.emotion_c)

In [7]:
df['nr_emotions'] = df.apply(lambda row: get_nr_emotions(row), axis=1)

In [8]:
df

,Unnamed: 0,file_name,page_nr,split,utterance,emotion_c,nr_emotions
0,0,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1,TRAIN,"[""THIS VILE THING ATTACKED THE SMALL BEASTS OF...","[['anger'], ['anger'], ['fear'], ['fear'], ['f...",137
1,1,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,2,TRAIN,"[""NO-- #GKKK\u2026#"", ""#CHOMP!"", ""BY THE SKIN...","[['fear'], ['anger'], ['surprise'], ['anger'],...",99
2,2,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,3,TRAIN,"[""COME ON, BEAST!"", ""SHOW YOURSELF!"", ""WHY DO ...","[['joy'], ['joy'], ['anger'], ['anger']]",40
3,3,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,4,TRAIN,"[""#AARGH! ""]","[['fear', 'surprise']]",22
4,4,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,5,TRAIN,"[""I, THE GREEN TORCH, HAVE BEEN TASKED WITH PR...","[['anger'], ['anger'], ['fear'], ['fear', 'sur...",83
...,...,...,...,...,...,...,...
869,869,QC copy - 2200 - Stillwater 13.xlsx,16,TEST,"[""WE WERE IN GALEN'S OFFICE. YOU WERE ABOUT TO...","[['anger'], ['anger'], ['anger'], ['anger'], [...",166
870,870,QC copy - 2200 - Stillwater 13.xlsx,17,TEST,"[""SO WHAT ARE WE GOING TO DO?"", ""THE WAY I SEE...","[['sadness', 'surprise'], ['anger'], ['anger']...",98
871,871,QC copy - 2200 - Stillwater 13.xlsx,18,TEST,"[""KIDDIE COUNCIL'S BEEN GOING A LONG TIME... ""...","[['anger', 'sadness'], ['anger'], ['anger'], [...",143
872,872,QC copy - 2200 - Stillwater 13.xlsx,19,TEST,"[""IT'S BEEN\u2026 PEACEFUL. ASIDE FROM SHIT LI...","[['anger'], ['joy'], ['joy'], ['anger', 'surpr...",144


In [9]:
df.file_name.unique()

array(['QC copy - 1499 - 58 ECC Co_mics 50 _The Jurassic League 4_.xlsx',
       'QC copy - 1500 - 04 Nightwing 19 _Nightwing 95_.xlsx',
       'QC copy - 1501 - 09 Mundos sin Liga de la Justicia Green Lantern 1 - DCWWJL GL 1.xlsx',
       'QC copy - 1502 - 09 Mundos sin Liga de la Justicia Green Lantern 1 - DCVMM Lobo.xlsx',
       'QC copy - 1503 - 10 Crisis Oscura Flash - FLS 783.xlsx',
       'QC copy - 1507 - 22 Calle Peligro 1.xlsx',
       'QC copy - 1508 - 48 Pequen_os Titanes 25.xlsx',
       'QC copy - 1513 - 21 Blanco Humano 9.xlsx',
       'QC copy - 1514 - 15 DC contra Vampiros 11.xlsx',
       'QC copy - 1517 - 37 John Carpenter Historias para una noche de Halloween 3.xlsx',
       'QC copy - 1518 - 50 Las asombrosas aventuras de las Tortugas Ninja 5.xlsx',
       'QC copy - 1520 - 51 Sonic The Hedgehog 44.xlsx',
       'QC copy - 1521 - 40 Amor eterno 2.xlsx',
       'QC copy - 1559 - 36 Fantasmas vol. 1 - Ghosted 2.xlsx',
       'QC copy - 1560 - 36 Fantasmas vol. 1 - G

In [10]:
df.nr_emotions.value_counts()

nr_emotions
116    11
44     10
156    10
123     9
109     9
       ..
313     1
236     1
256     1
187     1
233     1
Name: count, Length: 252, dtype: int64

In [11]:
file_names_l = df.file_name.unique().tolist()

In [12]:
comics_titles_full = [
    
    "Jurassic League #4",      
    "Nightwing #95",    
    "Dark Crisis: Worlds Without a Justice League - Green Lantern #1", 
    "Dark Crisis: Worlds Without a Justice League - Green Lantern - Very Merry Multiverse",
    "Dark Crisis: The Flash #783",
    "Danger Street #1",
    "Tiny Titans #25",
    "Human Target #9",
    "DC vs. Vampires #11",
    "John Carpenter's Tales for a Halloweenight #3",
    "The Amazing Adventures of the Ninja Turtles #5",
    "Sonic The Hedgehog #44",
    "Love Everlasting #2",
    "Fantasmas vol. 1 - Ghosted #2",
    "Fantasmas vol. 1 - Ghosted #3",
    "Fantasmas vol. 1 - Ghosted #4",
    "Fantasmas vol. 1 - Ghosted #5",
    "Fantasmas vol. 1 - Ghosted #1",
    "Fantasmas vol. 1 - Ghosted #6",
    "Fantasmas vol. 1 - Ghosted #7",
    "Fantasmas vol. 1 - Ghosted #8",
    "Fantasmas vol. 1 - Ghosted #9",
    "Fantasmas vol. 1 - Ghosted #10",
    "American Vampire vol. 4 - #6",
    "American Vampire vol. 4 - #7",
    "American Vampire vol. 4 - #8",
    "Dragon Age vol. 3 Engano - Deception #1",
    "Dragon Age vol. 3 Engano - Deception #2",
    "Dragon Age vol. 3 Engano - Deception #3",
    "The Walking Dead vol 15 - #169",
    "The Walking Dead vol 15 - #170",
    "The Walking Dead vol 15 - #171",
    "The Walking Dead vol 15 - #173",
    "Thief Of Thieves 1 - #11",
    "Stillwater #13",

]


In [13]:
len(comics_titles_full)

35

In [14]:
# comics_titles = [
    
#     "Nightwing",
#     "Worlds Without a Justice League - Green Lantern",
#     "Human Target",
#     "American Vampire",
#     "Dragon Age",
#     "Dragon Age",
#     "The Walking Dead",
#     "Worlds Without a Justice League - Green Lantern",
#     "Dark Crisis: The Flash",
#     "Danger Street",
#     "Tiny Titans",
#     "The Amazing Adventures of the Ninja Turtles",
#     "Sonic The Hedgehog",
#     "Love Everlasting",
#     "Fantasmas",
#     "Fantasmas",
#     "Fantasmas",
#     "Fantasmas",
#     "Fantasmas",
#     "Fantasmas",
#     "Fantasmas",
#     "Fantasmas",
#     "Fantasmas",
#     "Fantasmas",
#     "American Vampire",
#     "American Vampire",
#     "Dragon Age",
#     "Stillwater",
#     "Jurassic League",
#     "John Carpenter's Tales for a Halloweenight",
#     "DC vs. Vampires",
#     "Thief Of Thieves",
#     "The Walking Dead",
#     "The Walking Dead",
#     "The Walking Dead",

# ]


In [15]:
#len(comics_titles)

In [16]:
df['comics_title'] = df['file_name'].apply(lambda x: comics_titles_full[file_names_l.index(x)] if x in file_names_l else None)

In [17]:
df

,Unnamed: 0,file_name,page_nr,split,utterance,emotion_c,nr_emotions,comics_title
0,0,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1,TRAIN,"[""THIS VILE THING ATTACKED THE SMALL BEASTS OF...","[['anger'], ['anger'], ['fear'], ['fear'], ['f...",137,Jurassic League #4
1,1,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,2,TRAIN,"[""NO-- #GKKK\u2026#"", ""#CHOMP!"", ""BY THE SKIN...","[['fear'], ['anger'], ['surprise'], ['anger'],...",99,Jurassic League #4
2,2,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,3,TRAIN,"[""COME ON, BEAST!"", ""SHOW YOURSELF!"", ""WHY DO ...","[['joy'], ['joy'], ['anger'], ['anger']]",40,Jurassic League #4
3,3,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,4,TRAIN,"[""#AARGH! ""]","[['fear', 'surprise']]",22,Jurassic League #4
4,4,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,5,TRAIN,"[""I, THE GREEN TORCH, HAVE BEEN TASKED WITH PR...","[['anger'], ['anger'], ['fear'], ['fear', 'sur...",83,Jurassic League #4
...,...,...,...,...,...,...,...,...
869,869,QC copy - 2200 - Stillwater 13.xlsx,16,TEST,"[""WE WERE IN GALEN'S OFFICE. YOU WERE ABOUT TO...","[['anger'], ['anger'], ['anger'], ['anger'], [...",166,Stillwater #13
870,870,QC copy - 2200 - Stillwater 13.xlsx,17,TEST,"[""SO WHAT ARE WE GOING TO DO?"", ""THE WAY I SEE...","[['sadness', 'surprise'], ['anger'], ['anger']...",98,Stillwater #13
871,871,QC copy - 2200 - Stillwater 13.xlsx,18,TEST,"[""KIDDIE COUNCIL'S BEEN GOING A LONG TIME... ""...","[['anger', 'sadness'], ['anger'], ['anger'], [...",143,Stillwater #13
872,872,QC copy - 2200 - Stillwater 13.xlsx,19,TEST,"[""IT'S BEEN\u2026 PEACEFUL. ASIDE FROM SHIT LI...","[['anger'], ['joy'], ['joy'], ['anger', 'surpr...",144,Stillwater #13


In [18]:
### Get summary

In [ ]:
max_seq_length = 2048
summary_model, summary_tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/SmolLM2-1.7B-Instruct",
    #model_name="unsloth/Qwen2.5-7B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

In [21]:
summary_model = FastLanguageModel.for_inference(summary_model)

In [22]:
title_summaries_d = {}

for title in tqdm(comics_titles_full):
    
    summary_instruction = f"""You are an expert in comics with comprehensive knowledge of storylines, characters, and narrative arcs. When given a comics title, provide a thorough but concise summary that includes:

    1. The main plot from beginning to end
    2. Major story arcs and their resolutions
    3. Any significant themes or messages

    Important: Keep the summary focused and under 200 words while ensuring it has a proper conclusion. Do not leave any sentence unfinished.
    
    Please ensure the summary is cohesive and complete, with proper narrative flow from start to finish.
    Provide a well-structured summary that connects all plot points and gives closure to the story."""
    
    sys_msg = {'role': 'system', 'content': summary_instruction}
    user_msg = {'role': 'user', 'content': "Here is the comics title: \n" + title}
    assistant_msg = {'role': 'assistant', 'content': ""}
    
    messages = [sys_msg, user_msg, assistant_msg]
    
    input_text = summary_tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
    
    outputs = summary_model.generate(
        input_ids=input_text,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1
    )
    
    input_length = input_text.shape[1]
    generated_tokens = outputs[0][input_length:]
    #print(outputs)
    decoded_output = summary_tokenizer.decode(generated_tokens, skip_special_tokens=True)
    title_summaries_d[title] = decoded_output
    
    #break

100%|██████████| 35/35 [02:35<00:00,  4.46s/it]


In [23]:
title_summaries_d

{'Jurassic League #4': 'In the fourth issue of the "Jurassic League," the team embarks on a perilous journey to prevent the destruction of the ancient world by a rogue time traveler named "The Temporal Rogue." The team faces formidable challenges as they navigate through treacherous landscapes and battle formidable dinosaurs, including the fearsome Velociraptor and the intimidating Tyrannosaurus Rex.\n\nAs the story unfolds, the team discovers that the Temporal Rogue\'s actions are motivated by a personal tragedy - the loss of his own family during a catastrophic event in the past. This revelation sparks a complex emotional response within the team, leading to internal conflicts and power struggles.\n\nDespite these tensions, the team rallies together to face their greatest challenge yet - a massive T-Rex storming the camp after being awakened by the Temporal Rogue\'s actions. With the fate of the ancient world hanging in the balance, the team must confront their deepest fears and over

In [24]:
len(title_summaries_d)

35

In [25]:
df['comics_summary'] = df['comics_title'].map(title_summaries_d)

In [26]:
df

,Unnamed: 0,file_name,page_nr,split,utterance,emotion_c,nr_emotions,comics_title,comics_summary
0,0,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,1,TRAIN,"[""THIS VILE THING ATTACKED THE SMALL BEASTS OF...","[['anger'], ['anger'], ['fear'], ['fear'], ['f...",137,Jurassic League #4,"In the fourth issue of the ""Jurassic League,"" ..."
1,1,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,2,TRAIN,"[""NO-- #GKKK\u2026#"", ""#CHOMP!"", ""BY THE SKIN...","[['fear'], ['anger'], ['surprise'], ['anger'],...",99,Jurassic League #4,"In the fourth issue of the ""Jurassic League,"" ..."
2,2,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,3,TRAIN,"[""COME ON, BEAST!"", ""SHOW YOURSELF!"", ""WHY DO ...","[['joy'], ['joy'], ['anger'], ['anger']]",40,Jurassic League #4,"In the fourth issue of the ""Jurassic League,"" ..."
3,3,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,4,TRAIN,"[""#AARGH! ""]","[['fear', 'surprise']]",22,Jurassic League #4,"In the fourth issue of the ""Jurassic League,"" ..."
4,4,QC copy - 1499 - 58 ECC Co_mics 50 _The Jurass...,5,TRAIN,"[""I, THE GREEN TORCH, HAVE BEEN TASKED WITH PR...","[['anger'], ['anger'], ['fear'], ['fear', 'sur...",83,Jurassic League #4,"In the fourth issue of the ""Jurassic League,"" ..."
...,...,...,...,...,...,...,...,...,...
869,869,QC copy - 2200 - Stillwater 13.xlsx,16,TEST,"[""WE WERE IN GALEN'S OFFICE. YOU WERE ABOUT TO...","[['anger'], ['anger'], ['anger'], ['anger'], [...",166,Stillwater #13,"In the latest issue of Stillwater #13, our pro..."
870,870,QC copy - 2200 - Stillwater 13.xlsx,17,TEST,"[""SO WHAT ARE WE GOING TO DO?"", ""THE WAY I SEE...","[['sadness', 'surprise'], ['anger'], ['anger']...",98,Stillwater #13,"In the latest issue of Stillwater #13, our pro..."
871,871,QC copy - 2200 - Stillwater 13.xlsx,18,TEST,"[""KIDDIE COUNCIL'S BEEN GOING A LONG TIME... ""...","[['anger', 'sadness'], ['anger'], ['anger'], [...",143,Stillwater #13,"In the latest issue of Stillwater #13, our pro..."
872,872,QC copy - 2200 - Stillwater 13.xlsx,19,TEST,"[""IT'S BEEN\u2026 PEACEFUL. ASIDE FROM SHIT LI...","[['anger'], ['joy'], ['joy'], ['anger', 'surpr...",144,Stillwater #13,"In the latest issue of Stillwater #13, our pro..."


In [27]:
df_train = df[df.split == "TRAIN"].reset_index(drop=True)

In [28]:
len(df_train)

718

### Comics dataset

In [155]:
def build_instruction():
   
    emotion_classes = ["anger", "disgust", "fear", "sadness", "surprise", "joy", "neutral"]
    formatted_classes = ", ".join([f'"{emotion}"' for emotion in emotion_classes])
    
    instruction = f"""### Emotion Analysis Expert Role

You are an advanced emotion analysis expert specializing in comic book dialogue interpretation. Your task is to analyze utterances and identify their emotional content.

INPUT:
- You will receive a list of utterances from a page in a comic book
- The utterance may express one or multiple emotions
- The name and summary of the comic book

TASK:
1. Carefully analyze the emotional context and tone of each utterance in the page
2. Identify applicable emotions from the following classes:
   {formatted_classes}
3. For each utterance in a comic page, identify all emotions present and return an array of emotion arrays in order.

RULES:
1. Use ONLY the labels listed above
2. Output must be a JSON with single key "page_utterance_emotions"
3. Value must be an array where:
   - Each element is an array of emotions for one utterance
   - Order matches the input utterances order
   - Multiple emotions are allowed per utterance
4. No explanations, only JSON output

IMPORTANT:
- Each array element corresponds to one utterance
- One utterance can have multiple emotions
- Maintain exact spelling and case of emotion labels
- Keep emotions in arrays even for single emotions

"""
    return instruction

In [156]:
# def build_instruction():
  
#     emotion_classes = ["Anger", "Disgust", "Fear", "Sadness", "Surprise", "Joy", "Neutral"]
#     formatted_classes = ", ".join([f'"{emotion}"' for emotion in emotion_classes])
    
#     instruction = f"""### Emotion Analysis Expert Role

# You are an advanced emotion analysis expert specializing in comic book dialogue interpretation. Your task is to analyze utterances and identify their emotional content while considering conversational context.

# INPUT:
# - You will receive a list of 6 consecutive utterances from a comic book:
#   * The first 5 utterances provide conversational context
#   * The 6th (last) utterance is the one to be classified
# - Each utterance may express one or multiple emotions

# TASK:
# 1. Read through the context utterances to understand the emotional flow
# 2. Carefully analyze the emotional context, tone, and potential emotional shifts in the final utterance
# 3. Identify applicable emotions for the final utterance only from the following classes:
#    {formatted_classes}

# OUTPUT REQUIREMENTS:
# - Format: JSON object with a single key "list_emotion_classes"
# - Value: Array of one or more emotion classes as strings
# - The classification should be for the final utterance only, using previous utterances as context

# IMPORTANT NOTES:
# - Do not include any explanations in the output, only the JSON object
# - Use the context to better understand emotional transitions and current emotional state
# - Context utterances may reveal emotional buildup or shifts that influence the final utterance

# """
#     return instruction

In [157]:
instruction = build_instruction()

In [158]:
sys_msg_l = []
user_msg_l = []
assistant_msg_l = []

for _, row in df.iterrows():
        
        sys_msg = {'role': 'system', 'content': instruction}
        
        comics_title = row['comics_title']
        comics_summary = row['comics_summary']
        
        utterances_l = eval(row['utterance'])
        pg_utterances = "\n".join(f"{i+1}. {title}" for i, title in enumerate(utterances_l))
        
        usr_prompt = f"Comics title: {comics_title}\n" + f"Comics summary: {comics_summary}\n" + "Here is the list of utterances that you will classify: \n" + pg_utterances
        
        user_msg = {'role': 'user', 'content': usr_prompt}        
        assistant_msg = {'role': 'assistant', 'content': ""}


        sys_msg_l.append(sys_msg)
        user_msg_l.append(user_msg)
        assistant_msg_l.append(assistant_msg)
        

In [159]:
comics_dataset = []

for i in range(len(sys_msg_l)):

    comics_dataset.append([sys_msg_l[i], user_msg_l[i], assistant_msg_l[i]])

In [160]:
len(comics_dataset)

874

In [161]:
comics_dataset[0]

[{'role': 'system',
  'content': '### Emotion Analysis Expert Role\n\nYou are an advanced emotion analysis expert specializing in comic book dialogue interpretation. Your task is to analyze utterances and identify their emotional content.\n\nINPUT:\n- You will receive a list of utterances from a page in a comic book\n- The utterance may express one or multiple emotions\n- The name and summary of the comic book\n\nTASK:\n1. Carefully analyze the emotional context and tone of each utterance in the page\n2. Identify applicable emotions from the following classes:\n   "anger", "disgust", "fear", "sadness", "surprise", "joy", "neutral"\n3. For each utterance in a comic page, identify all emotions present and return an array of emotion arrays in order.\n\nRULES:\n1. Use ONLY the labels listed above\n2. Output must be a JSON with single key "page_utterance_emotions"\n3. Value must be an array where:\n   - Each element is an array of emotions for one utterance\n   - Order matches the input utt

In [162]:
model = FastLanguageModel.for_inference(model)

In [164]:
raw_outputs = []

for message in tqdm(comics_dataset):
    
    inputs = tokenizer.apply_chat_template(
    message,
    tokenize=True,
    add_generation_prompt=True,
    #return_dict=True,
    return_tensors="pt",
).to("cuda")
    #print(inputs)
    #break
    
    output = model.generate(input_ids=inputs, max_new_tokens=128)[0]
    #output = model.generate(**inputs, max_new_tokens=128)[0]
    
    input_length = inputs.shape[1]
    generated_tokens = output[input_length:]
    
    decoded_output = tokenizer.decode(generated_tokens, skip_special_tokens=True)  
    #decoded_output = tokenizer.decode(output, skip_special_tokens=True)
    raw_outputs.append(decoded_output)
    #break

100%|██████████| 874/874 [56:19<00:00,  3.87s/it]  


In [165]:
raw_outputs

['{\n    "page_utterance_emotions": [\n        [\n            "anger"\n        ],\n        [\n            "anger",\n            "disgust"\n        ],\n        [\n            "fear"\n        ],\n        [\n            "fear"\n        ],\n        [\n            "anger",\n            "disgust"\n        ],\n        [\n            "sadness"\n        ],\n        [\n            "fear",\n            "surprise"\n        ],\n        [\n            "surprise"\n        ],\n        [\n            "surprise"\n        ],\n        [\n            "fear"\n        ]\n    ]\n}',
 '{\n  "page_utterance_emotions": [\n    ["anger", "disgust"],\n    ["surprise", "joy"],\n    ["anger", "fear"],\n    ["anger", "fear", "surprise"],\n    ["surprise", "joy"],\n    ["disgust"],\n    ["neutral"],\n    ["anger"],\n    ["anger", "joy"]\n  ]\n}',
 '{\n  "page_utterance_emotions": [\n    ["anger"],\n    ["anger", "surprise"],\n    ["anger", "disgust"],\n    ["joy", "anger"]\n  ]\n}',
 '```json\n{\n  "page_utterance_emot

In [39]:
# def fix_comics_dataset(comics_dataset):
#     fixed_comics_dataset = []
#     for conversation in comics_dataset:
#         fixed_conversation = []
#         for message in conversation:
#             if isinstance(message['content'], list):  # If the 'value' is a list of emotions
#                 message['content'] = ', '.join(message['content'])  # Join the list into a string
#             fixed_conversation.append(message)
#         fixed_comics_dataset.append(fixed_conversation)
#     return fixed_comics_dataset

In [40]:
#fixed_comics_dataset = fix_comics_dataset(comics_dataset)

In [41]:
# dataset = Dataset.from_dict({
#     'conversations': fixed_comics_dataset
# })

In [42]:
#dataset

In [43]:
#dataset[0]['conversations']

In [44]:
# tokenizer = get_chat_template(
#     tokenizer,
#     mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
#     chat_template="chatml",
# )

# def apply_template_comics(examples):
#     messages = examples["conversations"]
#     #messages = examples['input'] + examples['output']
#     text = [tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=False) for message in messages]
#     return {"text": text}

In [45]:
#comics_dataset = dataset.map(apply_template_comics, batched=True)

In [46]:
#comics_dataset

In [47]:
# def split_dataset(dataset, train_ratio=0.8):
#     train_test = dataset.train_test_split(test_size=1 - train_ratio)
#     return train_test

# dataset_split = split_dataset(comics_dataset)

In [48]:
# train_dataset = dataset_split['train']
# eval_dataset = dataset_split['test']

In [49]:
# train_dataset

In [50]:
# print(train_dataset[452]['text'])

In [51]:
# OUTPUT_DIR = "/Utilisateurs/umushtaq/emotion_analysis_comics/outputs_dir_tmp"

In [52]:
# args=TrainingArguments(
#         learning_rate=3e-4,
#         lr_scheduler_type="cosine",
#         per_device_train_batch_size=16,
#         gradient_accumulation_steps=2,
#         num_train_epochs=10,
#         fp16=not is_bfloat16_supported(),
#         bf16=is_bfloat16_supported(),
#         logging_steps=25,
#         optim="adamw_8bit",
#         weight_decay=0.01,
#         warmup_steps=10,
        
#         eval_strategy="steps",  # Run evaluation during training (can also use "epoch")
#         eval_steps=25,  # Perform evaluation every 50 steps
#         save_strategy="steps",  # Save the model every few steps
#         save_steps=25,  # Save every 200 steps
#         load_best_model_at_end=True,
    
#         output_dir=OUTPUT_DIR,
#         seed=0,
#     )

In [53]:
# trainer=SFTTrainer(
#     model=model,
#     tokenizer=tokenizer,
#     train_dataset=train_dataset,  # Replace with your train dataset
#     eval_dataset=eval_dataset, 
#     dataset_text_field="text",
#     max_seq_length=max_seq_length,
#     dataset_num_proc=2,
#     packing=False,
#     args=args,
# )

In [54]:
# trainer.train()

In [55]:
#model = FastLanguageModel.for_inference(model)

In [56]:
#df_test = df[df.split == "TEST"].reset_index(drop=True)

In [57]:
# sys_msg_l = []
# user_msg_l = []
# assistant_msg_l = []

# for _, row in df_test.iterrows():
        
#         sys_msg = {'role': 'system', 'content': instruction}
        
        
#         comics_title = row['comics_title']
#         comics_summary = row['comics_summary']
        
#         utterances_l = eval(row['utterance'])
#         pg_utterances = "\n".join(f"{i+1}. {title}" for i, title in enumerate(utterances_l))
        
#         usr_prompt = f"Comics title: {comics_title}\n" + f"Comics summary: {comics_summary}\n" + "Here is the list of utterances that you will classify: \n" + pg_utterances
        
#         user_msg = {'role': 'user', 'content': usr_prompt}

#         #obj = {"list_emotion_classes": row['emotion_u']}
#         #obj = row['emotion_u']
#         #assistant_msg = {'from': 'gpt', 'value': obj}
#         assistant_msg = {'role': 'assistant', 'content': ""}


#         sys_msg_l.append(sys_msg)
#         user_msg_l.append(user_msg)
#         assistant_msg_l.append(assistant_msg)
        

In [58]:
# test_messages = []

# for i in range(len(sys_msg_l)):
    
#     #obj = {"list_emotion_classes": ["Anger", "Fear"]}

#     #comics_dataset.append([human_msg_l[i], assistant_msg_l[i]])
#     test_messages.append([sys_msg_l[i], user_msg_l[i], assistant_msg_l[i]])

In [59]:
# human_msg_l = []
# assistant_msg_l = []

# for _, row in df_test.iterrows():
        
#         prompt = instruction.replace("<comic_title>", row['comics_title']).replace("<speaker_id>", row['speaker_id']).replace("<utterance>", row['utterance'])
        
#         human_msg = {'role': 'user', 'content': prompt}
        
#         #obj = {"list_emotion_classes": row['emotion_u']}
#         obj = row['emotion_u']
#         assistant_msg = {'role': 'assistant', 'content': ""}
        
#         human_msg_l.append(human_msg)
#         assistant_msg_l.append(assistant_msg)
        
        

In [60]:
# test_messages = []

# for i in range(len(human_msg_l)):
    
#     #obj = {"list_emotion_classes": ["Anger", "Fear"]}

#     test_messages.append([human_msg_l[i], assistant_msg_l[i]])

In [61]:
#len(test_messages)

In [62]:
# test_messages = test_messages[:100]

In [63]:
# raw_outputs = []

# for message in tqdm(test_messages):
    
#     inputs = tokenizer.apply_chat_template(
#     message,
#     tokenize=True,
#     add_generation_prompt=True,
#     #return_dict=True,
#     return_tensors="pt",
# ).to("cuda")
#     #print(inputs)
#     #break
    
#     output = model.generate(input_ids=inputs, max_new_tokens=128)[0]
#     #output = model.generate(**inputs, max_new_tokens=128)[0]
    
#     input_length = inputs.shape[1]
#     generated_tokens = output[input_length:]
    
#     decoded_output = tokenizer.decode(generated_tokens, skip_special_tokens=True)  
#     #decoded_output = tokenizer.decode(output, skip_special_tokens=True)
#     raw_outputs.append(decoded_output)
#     #break

In [64]:
#print(tokenizer.decode(inputs[0]))

In [166]:
len(raw_outputs)

874

In [167]:
raw_outputs

['{\n    "page_utterance_emotions": [\n        [\n            "anger"\n        ],\n        [\n            "anger",\n            "disgust"\n        ],\n        [\n            "fear"\n        ],\n        [\n            "fear"\n        ],\n        [\n            "anger",\n            "disgust"\n        ],\n        [\n            "sadness"\n        ],\n        [\n            "fear",\n            "surprise"\n        ],\n        [\n            "surprise"\n        ],\n        [\n            "surprise"\n        ],\n        [\n            "fear"\n        ]\n    ]\n}',
 '{\n  "page_utterance_emotions": [\n    ["anger", "disgust"],\n    ["surprise", "joy"],\n    ["anger", "fear"],\n    ["anger", "fear", "surprise"],\n    ["surprise", "joy"],\n    ["disgust"],\n    ["neutral"],\n    ["anger"],\n    ["anger", "joy"]\n  ]\n}',
 '{\n  "page_utterance_emotions": [\n    ["anger"],\n    ["anger", "surprise"],\n    ["anger", "disgust"],\n    ["joy", "anger"]\n  ]\n}',
 '```json\n{\n  "page_utterance_emot

In [168]:
grounds = df.emotion_c.tolist()

In [169]:
len(grounds)

874

In [170]:
import json

In [171]:
predictions = [json_repair.loads(e) for e in raw_outputs]

In [172]:
predictions

[{'page_utterance_emotions': [['anger'],
   ['anger', 'disgust'],
   ['fear'],
   ['fear'],
   ['anger', 'disgust'],
   ['sadness'],
   ['fear', 'surprise'],
   ['surprise'],
   ['surprise'],
   ['fear']]},
 {'page_utterance_emotions': [['anger', 'disgust'],
   ['surprise', 'joy'],
   ['anger', 'fear'],
   ['anger', 'fear', 'surprise'],
   ['surprise', 'joy'],
   ['disgust'],
   ['neutral'],
   ['anger'],
   ['anger', 'joy']]},
 {'page_utterance_emotions': [['anger'],
   ['anger', 'surprise'],
   ['anger', 'disgust'],
   ['joy', 'anger']]},
 {'page_utterance_emotions': [['anger']]},
 {'page_utterance_emotions': [['joy'],
   ['anger', 'joy'],
   ['anger'],
   ['anger', 'fear'],
   ['fear', 'surprise'],
   ['joy']]},
 {'page_utterance_emotions': [['anger'],
   ['fear'],
   ['anger'],
   ['fear'],
   ['fear', 'surprise'],
   ['fear'],
   ['fear', 'anger'],
   ['fear', 'anger'],
   ['joy'],
   ['joy', 'anger']]},
 {'page_utterance_emotions': [['joy'],
   ['anger', 'disgust'],
   ['surprise

In [173]:
# #predictions = [e.split('\n\n')[0] for e in raw_outputs]
# bad_idx = []
# predictions = []

# for idx, e in enumerate(raw_outputs):
#     try:
#         predictions.append(json.loads(e))
#     except:
#         print(idx)
#         bad_idx.append(idx)
        


In [174]:
#len(predictions)

In [175]:
#predictions

In [176]:
#predictions = [json_repair.loads(e) for e in predictions]

In [177]:
#predictions

In [178]:
preds_l = []
bad_idx = []

for i, pred in enumerate(predictions):
    try:        
        preds_l.append(pred['page_utterance_emotions'])
    except:
        print(i)
        bad_idx.append(i)

In [179]:
preds_l

[[['anger'],
  ['anger', 'disgust'],
  ['fear'],
  ['fear'],
  ['anger', 'disgust'],
  ['sadness'],
  ['fear', 'surprise'],
  ['surprise'],
  ['surprise'],
  ['fear']],
 [['anger', 'disgust'],
  ['surprise', 'joy'],
  ['anger', 'fear'],
  ['anger', 'fear', 'surprise'],
  ['surprise', 'joy'],
  ['disgust'],
  ['neutral'],
  ['anger'],
  ['anger', 'joy']],
 [['anger'], ['anger', 'surprise'], ['anger', 'disgust'], ['joy', 'anger']],
 [['anger']],
 [['joy'],
  ['anger', 'joy'],
  ['anger'],
  ['anger', 'fear'],
  ['fear', 'surprise'],
  ['joy']],
 [['anger'],
  ['fear'],
  ['anger'],
  ['fear'],
  ['fear', 'surprise'],
  ['fear'],
  ['fear', 'anger'],
  ['fear', 'anger'],
  ['joy'],
  ['joy', 'anger']],
 [['joy'], ['anger', 'disgust'], ['surprise'], ['surprise', 'joy'], ['joy']],
 [['anger', 'fear'],
  ['fear', 'surprise'],
  ['anger', 'confusion'],
  ['joy', 'determination'],
  ['anger', 'defiance'],
  ['fear', 'uncertainty'],
  ['fear', 'caution'],
  ['anger', 'frustration'],
  ['anger',

In [180]:
grounds = [item for i, item in enumerate(grounds) if i not in bad_idx]

In [181]:
len(grounds), len(preds_l)

(874, 874)

In [182]:
preds_l

[[['anger'],
  ['anger', 'disgust'],
  ['fear'],
  ['fear'],
  ['anger', 'disgust'],
  ['sadness'],
  ['fear', 'surprise'],
  ['surprise'],
  ['surprise'],
  ['fear']],
 [['anger', 'disgust'],
  ['surprise', 'joy'],
  ['anger', 'fear'],
  ['anger', 'fear', 'surprise'],
  ['surprise', 'joy'],
  ['disgust'],
  ['neutral'],
  ['anger'],
  ['anger', 'joy']],
 [['anger'], ['anger', 'surprise'], ['anger', 'disgust'], ['joy', 'anger']],
 [['anger']],
 [['joy'],
  ['anger', 'joy'],
  ['anger'],
  ['anger', 'fear'],
  ['fear', 'surprise'],
  ['joy']],
 [['anger'],
  ['fear'],
  ['anger'],
  ['fear'],
  ['fear', 'surprise'],
  ['fear'],
  ['fear', 'anger'],
  ['fear', 'anger'],
  ['joy'],
  ['joy', 'anger']],
 [['joy'], ['anger', 'disgust'], ['surprise'], ['surprise', 'joy'], ['joy']],
 [['anger', 'fear'],
  ['fear', 'surprise'],
  ['anger', 'confusion'],
  ['joy', 'determination'],
  ['anger', 'defiance'],
  ['fear', 'uncertainty'],
  ['fear', 'caution'],
  ['anger', 'frustration'],
  ['anger',

In [183]:
grounds

["[['anger'], ['anger'], ['fear'], ['fear'], ['fear', 'sadness'], ['sadness'], ['anger'], ['surprise'], ['surprise'], ['fear', 'surprise']]",
 "[['fear'], ['anger'], ['surprise'], ['anger'], ['joy'], ['anger'], ['anger'], ['anger'], ['anger']]",
 "[['joy'], ['joy'], ['anger'], ['anger']]",
 "[['fear', 'surprise']]",
 "[['anger'], ['anger'], ['fear'], ['fear', 'surprise'], ['anger', 'joy'], ['anger']]",
 "[['sadness'], ['fear'], ['joy'], ['neutral'], ['surprise'], ['fear'], ['surprise'], ['joy'], ['surprise'], ['sadness', 'surprise']]",
 "[['surprise'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['joy']]",
 "[['fear'], ['fear'], ['anger'], ['fear'], ['anger', 'surprise'], ['fear'], ['fear'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['anger'], ['anger']]",
 "[['fear'], ['fear'], ['anger'], ['anger', 'fear', 'surprise'], ['fear', 'sadness'], ['fear', 'sadness'], ['anger'], ['anger'], ['surprise'], ['fear'], ['anger', 'surprise'], ['anger'], ['anger', 'sadness'], ['anger']]",

In [184]:
import ast

grounds = [ast.literal_eval(x) for x in grounds]

In [185]:
grounds

[[['anger'],
  ['anger'],
  ['fear'],
  ['fear'],
  ['fear', 'sadness'],
  ['sadness'],
  ['anger'],
  ['surprise'],
  ['surprise'],
  ['fear', 'surprise']],
 [['fear'],
  ['anger'],
  ['surprise'],
  ['anger'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger']],
 [['joy'], ['joy'], ['anger'], ['anger']],
 [['fear', 'surprise']],
 [['anger'],
  ['anger'],
  ['fear'],
  ['fear', 'surprise'],
  ['anger', 'joy'],
  ['anger']],
 [['sadness'],
  ['fear'],
  ['joy'],
  ['neutral'],
  ['surprise'],
  ['fear'],
  ['surprise'],
  ['joy'],
  ['surprise'],
  ['sadness', 'surprise']],
 [['surprise'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['joy']],
 [['fear'],
  ['fear'],
  ['anger'],
  ['fear'],
  ['anger', 'surprise'],
  ['fear'],
  ['fear'],
  ['fear', 'sadness'],
  ['surprise'],
  ['surprise'],
  ['anger'],
  ['anger']],
 [['fear'],
  ['fear'],
  ['anger'],
  ['anger', 'fear', 'surprise'],
  ['fear', 'sadness'],
  ['fear', 'sadness'],
  ['anger'],
  ['anger'],
  ['surpris

In [186]:
preds_l

[[['anger'],
  ['anger', 'disgust'],
  ['fear'],
  ['fear'],
  ['anger', 'disgust'],
  ['sadness'],
  ['fear', 'surprise'],
  ['surprise'],
  ['surprise'],
  ['fear']],
 [['anger', 'disgust'],
  ['surprise', 'joy'],
  ['anger', 'fear'],
  ['anger', 'fear', 'surprise'],
  ['surprise', 'joy'],
  ['disgust'],
  ['neutral'],
  ['anger'],
  ['anger', 'joy']],
 [['anger'], ['anger', 'surprise'], ['anger', 'disgust'], ['joy', 'anger']],
 [['anger']],
 [['joy'],
  ['anger', 'joy'],
  ['anger'],
  ['anger', 'fear'],
  ['fear', 'surprise'],
  ['joy']],
 [['anger'],
  ['fear'],
  ['anger'],
  ['fear'],
  ['fear', 'surprise'],
  ['fear'],
  ['fear', 'anger'],
  ['fear', 'anger'],
  ['joy'],
  ['joy', 'anger']],
 [['joy'], ['anger', 'disgust'], ['surprise'], ['surprise', 'joy'], ['joy']],
 [['anger', 'fear'],
  ['fear', 'surprise'],
  ['anger', 'confusion'],
  ['joy', 'determination'],
  ['anger', 'defiance'],
  ['fear', 'uncertainty'],
  ['fear', 'caution'],
  ['anger', 'frustration'],
  ['anger',

In [187]:
bad_idx = []

for idx, (i,j) in enumerate(zip(grounds, preds_l)):
    if len(i) != len(j):
        print(idx, len(i), len(j))
        bad_idx.append(idx)

8 14 9
14 11 5
19 8 9
20 9 8
23 12 9
25 6 8
26 14 7
28 10 9
42 13 11
47 11 10
60 8 7
72 13 12
73 15 8
75 18 13
76 16 11
77 16 12
80 8 9
82 13 12
84 11 10
87 10 9
99 11 12
101 16 12
102 21 13
104 20 13
105 15 13
106 16 12
107 12 11
113 23 9
114 23 11
115 13 11
117 13 10
118 14 12
120 11 12
121 11 10
124 13 11
126 17 10
130 9 10
131 12 11
137 12 11
144 2 3
148 12 11
153 13 5
160 13 7
161 18 9
166 12 8
167 13 8
172 11 10
175 6 4
181 10 9
183 17 14
193 11 10
198 18 20
202 12 10
205 22 18
214 11 10
216 15 12
226 11 10
230 15 14
232 17 10
233 13 10
234 12 11
241 11 10
242 12 10
247 16 14
248 15 14
249 17 13
252 9 8
253 16 13
254 17 14
268 6 5
273 19 13
274 23 19
275 13 12
278 15 14
283 25 12
288 15 10
291 17 13
292 15 8
293 11 10
297 10 9
301 31 9
305 12 11
312 16 13
317 15 9
318 13 10
319 12 9
320 19 10
321 17 11
323 14 11
325 14 12
328 8 9
329 11 9
331 2 5
332 4 5
334 5 8
336 2 5
338 4 5
339 4 5
341 4 6
343 12 10
351 14 10
356 9 10
360 16 13
363 12 10
364 12 10
368 15 10
381 11 10
404 12 1

In [188]:
bad_idx.sort(reverse=True)

# Remove elements from 'grounds' at the specified indices
for idx in bad_idx:
    
    del grounds[idx]
    del preds_l[idx]

In [189]:
preds_l

[[['anger'],
  ['anger', 'disgust'],
  ['fear'],
  ['fear'],
  ['anger', 'disgust'],
  ['sadness'],
  ['fear', 'surprise'],
  ['surprise'],
  ['surprise'],
  ['fear']],
 [['anger', 'disgust'],
  ['surprise', 'joy'],
  ['anger', 'fear'],
  ['anger', 'fear', 'surprise'],
  ['surprise', 'joy'],
  ['disgust'],
  ['neutral'],
  ['anger'],
  ['anger', 'joy']],
 [['anger'], ['anger', 'surprise'], ['anger', 'disgust'], ['joy', 'anger']],
 [['anger']],
 [['joy'],
  ['anger', 'joy'],
  ['anger'],
  ['anger', 'fear'],
  ['fear', 'surprise'],
  ['joy']],
 [['anger'],
  ['fear'],
  ['anger'],
  ['fear'],
  ['fear', 'surprise'],
  ['fear'],
  ['fear', 'anger'],
  ['fear', 'anger'],
  ['joy'],
  ['joy', 'anger']],
 [['joy'], ['anger', 'disgust'], ['surprise'], ['surprise', 'joy'], ['joy']],
 [['anger', 'fear'],
  ['fear', 'surprise'],
  ['anger', 'confusion'],
  ['joy', 'determination'],
  ['anger', 'defiance'],
  ['fear', 'uncertainty'],
  ['fear', 'caution'],
  ['anger', 'frustration'],
  ['anger',

In [190]:
grounds = [item for sublist in grounds for item in sublist]
predictions = [item for sublist in preds_l for item in sublist]

In [191]:
len(grounds), len(predictions)

(4336, 4336)

In [192]:
grounds

[['anger'],
 ['anger'],
 ['fear'],
 ['fear'],
 ['fear', 'sadness'],
 ['sadness'],
 ['anger'],
 ['surprise'],
 ['surprise'],
 ['fear', 'surprise'],
 ['fear'],
 ['anger'],
 ['surprise'],
 ['anger'],
 ['joy'],
 ['anger'],
 ['anger'],
 ['anger'],
 ['anger'],
 ['joy'],
 ['joy'],
 ['anger'],
 ['anger'],
 ['fear', 'surprise'],
 ['anger'],
 ['anger'],
 ['fear'],
 ['fear', 'surprise'],
 ['anger', 'joy'],
 ['anger'],
 ['sadness'],
 ['fear'],
 ['joy'],
 ['neutral'],
 ['surprise'],
 ['fear'],
 ['surprise'],
 ['joy'],
 ['surprise'],
 ['sadness', 'surprise'],
 ['surprise'],
 ['fear', 'sadness'],
 ['surprise'],
 ['surprise'],
 ['joy'],
 ['fear'],
 ['fear'],
 ['anger'],
 ['fear'],
 ['anger', 'surprise'],
 ['fear'],
 ['fear'],
 ['fear', 'sadness'],
 ['surprise'],
 ['surprise'],
 ['anger'],
 ['anger'],
 ['joy'],
 ['joy'],
 ['fear'],
 ['joy'],
 ['fear', 'sadness'],
 ['anger', 'disgust', 'surprise'],
 ['anger', 'disgust'],
 ['fear'],
 ['anger', 'disgust'],
 ['fear', 'surprise'],
 ['anger'],
 ['joy'],
 ['j

In [193]:
predictions

[['anger'],
 ['anger', 'disgust'],
 ['fear'],
 ['fear'],
 ['anger', 'disgust'],
 ['sadness'],
 ['fear', 'surprise'],
 ['surprise'],
 ['surprise'],
 ['fear'],
 ['anger', 'disgust'],
 ['surprise', 'joy'],
 ['anger', 'fear'],
 ['anger', 'fear', 'surprise'],
 ['surprise', 'joy'],
 ['disgust'],
 ['neutral'],
 ['anger'],
 ['anger', 'joy'],
 ['anger'],
 ['anger', 'surprise'],
 ['anger', 'disgust'],
 ['joy', 'anger'],
 ['anger'],
 ['joy'],
 ['anger', 'joy'],
 ['anger'],
 ['anger', 'fear'],
 ['fear', 'surprise'],
 ['joy'],
 ['anger'],
 ['fear'],
 ['anger'],
 ['fear'],
 ['fear', 'surprise'],
 ['fear'],
 ['fear', 'anger'],
 ['fear', 'anger'],
 ['joy'],
 ['joy', 'anger'],
 ['joy'],
 ['anger', 'disgust'],
 ['surprise'],
 ['surprise', 'joy'],
 ['joy'],
 ['anger', 'fear'],
 ['fear', 'surprise'],
 ['anger', 'confusion'],
 ['joy', 'determination'],
 ['anger', 'defiance'],
 ['fear', 'uncertainty'],
 ['fear', 'caution'],
 ['anger', 'frustration'],
 ['anger', 'determination'],
 ['anger', 'surprise'],
 ['a

In [194]:
mlb = MultiLabelBinarizer()

In [195]:
y_true_mhot = mlb.fit_transform(grounds)
y_pred_mhot = mlb.transform(predictions)

/Utilisateurs/umushtaq/.conda/envs/triton_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['admiration', 'aggression', 'agreement', 'anticipation', 'apologetic', 'appreciation', 'approval', 'assertion', 'calm', 'caution', 'comfort', 'concern', 'confidence', 'confusion', 'curiosity', 'defensiveness', 'defiance', 'determination', 'doubt', 'duty', 'encouragement', 'error', 'f', 'friendship', 'frustration', 'gratitude', 'guilt', 'hope', 'humor', 'investigation', 'love', 'memory', 'neutrality', 'pain', 'pleasure', 'politeness', 'pride', 'reassurance', 'relief', 'sarcasm', 'sarcastic', 'skepticism', 'sur', 'suspicion', 'trust', 'uncertainty', 'urgency'] will be ignored
  warnings.warn(


In [196]:
y_pred_mhot.shape

(4336, 7)

In [197]:
y_pred_mhot.shape

(4336, 7)

In [198]:
print(classification_report(y_true_mhot, y_pred_mhot, target_names=mlb.classes_, digits=3))

              precision    recall  f1-score   support

       anger      0.524     0.565     0.544      1534
     disgust      0.150     0.335     0.207       212
        fear      0.401     0.434     0.417      1118
         joy      0.476     0.450     0.463      1035
     neutral      0.104     0.349     0.160       238
     sadness      0.473     0.235     0.314       991
    surprise      0.402     0.361     0.380      1141

   micro avg      0.395     0.417     0.406      6269
   macro avg      0.361     0.390     0.355      6269
weighted avg      0.435     0.417     0.416      6269
 samples avg      0.401     0.426     0.389      6269



/Utilisateurs/umushtaq/.conda/envs/triton_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
